In [2]:
import sys
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T
import logging
from gym.envs.registration import register
import scipy
from easydl import clear_output

[easydl] tensorflow not available!


In [3]:
import math
import meep as mp
import numpy as np
from meep import mpb
import meep.materials
import random
import gym
from gym import spaces, logger
from gym import utils
from gym.utils import seeding
from collections import namedtuple, deque
from itertools import count
import subprocess, time, signal
import numpy as np
import sys
import gc
import csv

In [3]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [ ]:
#### 两组离散数据（同X）求交点 ###
def numpy_get_roots_with_two_piecewise_linear_by_same_X(Y1 , Y2 , X , plot = 0):
    # 注1：Y1与Y2长度相等，每条直线段最多只能有一个交点（不可能有两个）
    # 注1：X的值必须由小到大（X须为单调递增序列）
    if X == None:
        X = [i+1 for i in range(len(Y1))]
    else:
        pass
    
    roots = []
    for i in range(len(Y1)-1):
        Y_i_1 = [Y1[i], Y1[i+1]]
        Y_i_2 = [Y2[i], Y2[i+1]]
        X_i = [X[i], X[i+1]]

        poly_i_1 = np.poly1d(np.polyfit(X_i, Y_i_1, 1))
        poly_i_2 = np.poly1d(np.polyfit(X_i, Y_i_2, 1))
        poly_delta = poly_i_1 - poly_i_2
        
        # 非水平直线函数poly_delta必定有且只有一个解
        root_i = poly_delta.roots[0]
        y = poly_i_1(root_i)
        
        # 减少精度，找回因数据精度损失造成节点处丢失的根
        root_i = round(root_i, 6)
        y = round(y, 6)
        #print((root_i, y), X_i)
        if root_i >=  X_i[0] and root_i <= X_i[1]:
            #print((root_i, y), X_i)
            roots.append(root_i)

    roots = list(set(roots))
    roots.sort()
    return roots

In [ ]:
class mpbRl():
    def _init_(self):
        pass
    
    def adjustdesignparams(self,dy,dx,Ra,Rb):
        num_bands = 15
        resolution = 16
        r=0.34
        geometry = []
        geometry_lattice = mp.Lattice(size=mp.Vector3(1,np.sqrt(3)*6))
        #colomn 1 btm
        for i in range(2):
            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i,-3*np.sqrt(3)),  material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i,-2*np.sqrt(3)),  material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i, -1*np.sqrt(3)),material=mp.air))
        #colomn 1 up

            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i, 1*np.sqrt(3)), material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i,2*np.sqrt(3)) , material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(-1/2+i,3*np.sqrt(3)) , material=mp.air))
        #colomn 2 btm
        for j in range(1):
            geometry.append(mp.Cylinder(r, center=mp.Vector3(0,-5/2*np.sqrt(3)), material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(0,-3/2*np.sqrt(3)), material=mp.air))
            geometry.append(mp.Ellipsoid(center=mp.Vector3(dx,-1/2*np.sqrt(3)+dy),size=mp.Vector3(2*(Ra+r),2*(Rb+r),mp.inf),e1=mp.Vector3(1,0),e2=mp.Vector3(0,1),
                                         material=mp.air))
        #colomn 2 up
            geometry.append(mp.Ellipsoid(center=mp.Vector3(dx,1*np.sqrt(3)/2+dy),size=mp.Vector3(2*(Ra+r),2*(Rb+r),mp.inf), e1=mp.Vector3(1,0),e2=mp.Vector3(0,1),
                                         material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(0,3/2*np.sqrt(3)),material=mp.air))
            geometry.append(mp.Cylinder(r, center=mp.Vector3(0,5/2*np.sqrt(3)),material=mp.air))
        #     geometry.append(mp.Ellipsoid(center=mp.Vector3(w1_0+k*d,0.7*d), size=mp.Vector3(2*s_a,2*s_b,0.01),
        #                e1=mp.Vector3(1,0),e2=mp.Vector3(0,1),material=mp.air))
        #     geometry.append(mp.Ellipsoid(center=mp.Vector3(w1_0+k*d,-1/2*np.sqrt(3)*d), size=mp.Vector3(2*s_a,2*s_b,0.01),
        #                 e1=mp.Vector3(1,0),e2=mp.Vector3(0,1),material=mp.air))


        #geometry.append(mp.Block(center=mp.Vector3(w1_0,1/2*np.sqrt(3)*d), size=mp.Vector3(s_a,s_b),
                       #material=mp.Medium(epsilon=2.83)))
        #geometry.append(mp.Block(center=mp.Vector3(w1_0,-1/2*np.sqrt(3)*d), size=mp.Vector3(s_a,s_b),
                        #material=mp.Medium(epsilon=2.83)))
        k_points = [
            #mp.Vector3(),                           # Gamma
            mp.Vector3(0.26, 0),   
            mp.Vector3(1/2, 0),          # K
            #mp.Vector3(0.58, 0),   
            #mp.Vector3(1, 0),          # Gamma
        ]
        k_points = mp.interpolate(54, k_points)

        ms = mpb.ModeSolver(
            geometry=geometry,
            geometry_lattice=geometry_lattice,
            k_points=k_points,
            resolution=resolution,
            num_bands=num_bands,
            default_material=mp.Medium(index=2.83)
        )
        hfields = []

        def get_hfields(ms, band):
            hfields.append(ms.get_hfield(band, bloch_phase=True))

        ms.run_te(mpb.output_at_kpoint(mp.Vector3(0, 1/2), mpb.fix_hfield_phase,
                  get_hfields))
        te_freqs = ms.all_freqs
        te_gaps = ms.gap_list
        
        
        te_freq=te_freqs[:,12]
        te_freq_l=np.argsort(-te_freq)
        te_freq=te_freq[te_freq_l]
        k=np.linspace(0.26,0.5,len(te_freq))
        diff_x = []                       # 存储x列表中的两数之差
        for i in range(len(te_freq)-1):
            diff = abs(te_freq[i+1]-te_freq[i])
            diff_x.append(diff)
        diff_y = []                       # 存储y列表中的两数之差
        for i in range(len(te_freq)-1):
            diff = abs(k[i+1]-k[i])
            diff_y.append(diff)
        slopes = []                       # 存储斜率
        for i in range(len(diff_y)):
            slopes.append(diff_y[i] / diff_x[i])
        #slopes
        # deriv = []                        # 存储一阶导数
        # for i, j in zip(slopes[0::], slopes[1::]):        
        #     deriv.append((j/i)) # 根据离散点导数的定义，计算并存储结果
        # deriv.insert(0, slopes[0])        # (左)端点的导数即为与其最近点的斜率
        # deriv.append(slopes[-1])          # (右)端点的导数即为与其最近点的斜率
        diff_w = []                       # 存储x列表中的两数之差
        for i in range(len(te_freq)-1):
            diff = 412/abs(te_freq[i])
            diff_w.append(diff)
        wavelength=[]
        ng=[]
        for i in range(len(te_freq)-2):
            if abs(slopes[i+1]-slopes[i])<=50 and slopes[i]>=10:
                wavelength.append(diff_w[i])
                ng.append(slopes[i])
            elif abs(slopes[i+1]-slopes[i])>50 and slopes[i]>=10:
                break
        ng_array=np.array(ng)
        ng_array=np.array(ng)
        peak_indexes = scipy.signal.argrelextrema(ng_array, np.greater, order=1)
        peak_indexes = peak_indexes[0]
        valley_indexes = scipy.signal.argrelextrema(ng_array, np.less, order=1)
        valley_indexes = valley_indexes[0]#求极值点
        if peak_indexes.size==0:
            root1=[]
            peak_value=0
        if peak_indexes.size==1:
            peak_value=ng[int(peak_indexes)]
            high_line=[peak_value]*len(ng)
            root1= numpy_get_roots_with_two_piecewise_linear_by_same_X(Y1=high_line,Y2=ng,X=wavelength)
        elif peak_indexes.size>1:
            peak_value=ng[peak_indexes]
            peak_value=max(peak_value)
            high_line=[peak_value]*len(ng)
            root1= numpy_get_roots_with_two_piecewise_linear_by_same_X(Y1=high_line,Y2=ng,X=wavelength)
        # Plot valleys
        if valley_indexes.size==0:
            root2=[]
            valley_value=0
        if valley_indexes.size==1:
            valley_value=ng[int(valley_indexes)]
            low_line=[valley_value]*len(ng)
            root2= numpy_get_roots_with_two_piecewise_linear_by_same_X(Y1=low_line,Y2=ng,X=wavelength)
        elif valley_indexes.size>1:
            valley_value=ng[valley_indexes]
            valley_value=min(valley_value)
            low_line=[valley_value]*len(ng)
            root2= numpy_get_roots_with_two_piecewise_linear_by_same_X(Y1=low_line,Y2=ng,X=wavelength)


        ng_value=(peak_value+valley_value)/2
        if len(root1)==0 and len(root2)==0:
            bandwidth=0
            NDBP=0
            ng_value_average=0
        elif len(root1)==0 and len(root2)!=0:
            bandwidth=root2[0]-0
            NDBP=0
            ng_value_average=0
        elif len(root1)!=0 and len(root2)==0:
            bandwidth=root1[-1]-0
            NDBP=0
            ng_value_average=0
        else:
            bandwidth=root1[-1]-root2[0]
            w1=412/root1[-1]
            w2=412/root2[0]
            w1_ng,w1_index=find_nearest(te_freq,w1)
            w2_ng,w2_index=find_nearest(te_freq,w2)
            ng_value_list=[]
            for w_h,w_l,ng_w in zip(te_freq[w2_index:w1_index+1],te_freq[w2_index+1:w1_index+2],slopes[w2_index:w1_index+1]):
                ng_value_list.append(ng_w*(w_h-w_l))
            ng_value_average=sum(ng_value_list)/(w2_ng-w1_ng)
            NDBP=ng_value_average*(te_freq[w2_index]-te_freq[w1_index])/(te_freq[w2_index]+te_freq[w1_index])*2
            gc.collect()
        return NDBP,ng_value_average,bandwidth

In [ ]:
FR=mpbRl()
dy=0.13
dx=0.027
Ra=-0.039
Rb=-0.095
par=FR.adjustdesignparams(dy,dx,Ra,Rb)
print(par)

In [ ]:
class mpbEnv(gym.Env):
    metadata = {'render.modes':['human']}

    def __init__(self):
        # limits for net geometrical changes (states)
        self.maxdy = 0.185
        self.maxdx = 0.5
        self.maxRa = 0.3
        self.maxRb = 0.3
        # actions to take (i.e. alter the geometrical parameters)
        # self.delta = 0.5e-9
        # self.DR = 0.25e-9

        self.deltay = 0.001
        self.deltax = 0.0025
        self.deltaRa = 0.002
        self.deltaRb = 0.002

        high = np.array(
            [
                self.maxdy * 1.5,
                self.maxdx * 1.5,
                self.maxRa * 1.5,
                self.maxRb * 1.5,
            ],
            dtype=np.float32,
        )

        self.action_space = spaces.Discrete(9)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        # best geometrical shift values found so far
        self.dy_optim = 0.16
        self.dx_optim = 0.02
        self.Ra_optim = -0.04
        self.Rb_optim = -0.12
        
        self.goal_ng = 70  # optimization goal
        self.goal_NDBP=0.8
        self.goal_bandwidth =20
        self.seed()
        self.viewer = None
        self.state = None
        self.steps_beyond_done = None
    def step(self,action):
        err_msg = "%r (%s) invalid" % (action, type(action))
        assert self.action_space.contains(action),err_msg

        netdy, netdx, netRa,netRb = self.state

        if action == 0:
            netdy = netdy + self.deltay

        elif action == 1:
            netdy = netdy - self.deltay

        elif action == 2:
            netdx = netdx + self.deltax

        elif action == 3:
            netdx = netdx - self.deltax

        elif action == 4:
            netRa = netRa + self.deltaRa

        elif action == 5:
            netRa = netRa - self.deltaRa
        
        elif action == 6:
            netRb = netRb + self.deltaRb

        elif action == 7:
            netRb = netRb - self.deltaRb
            
        elif action == 8:
            pass

        # perform an action in fdtd and compute Q factor
        SL = mpbRl()
        # define conversion from m to nm
        NDBP,ng,bandwidth= SL.adjustdesignparams(netdy, netdx, netRa,netRb)

        # update the state
        self.state = (netdy, netdx, netRa,netRb)

        done = bool(
            netdx < -self.maxdx
            or netdx > self.maxdx
            or netdy < -self.maxdy
            or netdy > self.maxdy
            or netRa < -self.maxRa
            or netRa > self.maxRa
            or netRb < -self.maxRb
            or netRb > self.maxRb
        )

        # calculate the reward
        if not done:
            r = (100 - (self.goal_NDBP - NDBP)*100)
            reward = np.float32(r)
        elif self.steps_beyond_done is None:
            # net changes out of limit, game over
            self.steps_beyond_done = 0
            r = (100 - (self.goal_NDBP - NDBP)*100)
            reward = np.float32(r)
            print('State out of range, done! Restarting a new episode...')
        else:
            if self.steps_beyond_done == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state, dtype=np.float32), reward, done, {}

    def reset(self):

        # self.state = np.zeros((3,), dtype=np.float32)
        self.state = (self.dy_optim, self.dx_optim, self.Ra_optim,self.Rb_optim)
        self.steps_beyond_done = None
        return np.array(self.state, dtype=np.float32)


In [13]:
torch.set_printoptions(precision=4)

logger = logging.getLogger(__name__)

# register the env with gym
register(
    id='mpbEnv-v0',
    entry_point='envs:mpbEnv',
    max_episode_steps=1000,
    reward_threshold=80.0,
)

writer = SummaryWriter()  # log the training process

# instantiate the fdtd env
env = gym.make('mpbEnv-v0').unwrapped

# if GPU is to be used
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# declare transition and experience replay
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    """declare the replay buffer"""

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
# set up the neural network
# create a class for the DQN's policy MLP states predict actions
class Net(nn.Module):
    def __init__(self, num_actions):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 64)  # just FC, no CNN
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_actions)

    def forward(self, x):
        x = x.to(device)
        #         print(x.shape)
        x = x.view(-1, 5)#改变形状
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x


env.reset()

# set up the training
BATCH_SIZE = 40
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 5
UPDATE_FREQ = 1
#选择随机动作的概率将从 EPS_START 开始并以指数形式向 EPS_END衰减。 EPS_DECAY 控制衰减速率。
# get number of actions from gym action space
n_actions = env.action_space.n

policy_net = Net(n_actions).to(device)  # instantiate the policy network
target_net = Net(n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters(), lr=0.00025, alpha=0.95, momentum=0.95)  # initialize the optimizer, change learning rate?
# optimizer = optim.Adam(policy_net.parameters(), lr=0.00025)

memory = ReplayMemory(80000)  # instantiate the replay buffer

steps_done = 0  # counter for steps taken
def select_action(state):
    """selects an action accordingly to an epsilon greedy policy"""
    global steps_done
    sample = random.random()  # generate random number
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)  # expotentially decaying eps
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            print(policy_net(state))
            print(policy_net(state).max(1)[1])
            return policy_net(state).max(1)[1].view(1, 1)  # Pick action with the largest expected reward (argmax)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)  # pick random action

# define the optimization (RL) process, which computes V, Q and the loss
def optimize_model():

    if len(memory) < BATCH_SIZE:
        return

    print('optimizing...')

    transitions = memory.sample(BATCH_SIZE)  # sample transitions from the replay buffer
    batch = Transition(*zip(*transitions))  # transpose the batch

    # compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device,
                                  dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    # state, action, and reward from replay buffer
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # compute Q(s, a)
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    # Compute V(s')
    next_state_values = torch.zeros(BATCH_SIZE, device=device)  # V is zero for final state
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()  # V' = max(Q')
    # compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch  # Q_expected = r + gamma*V'

    # cost function
    criterion = nn.SmoothL1Loss()#损失函数：nn.L1Loss、nn.SmoothL1Loss、nn.MSELoss 、nn.CrossEntropyLoss、nn.NLLLoss
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))  # L = Q.actual - Q.expected

    # optimize the MLP model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        # clamp grad values to between -1 and 1
        param.grad.data.clamp_(-1,1)
    optimizer.step()
    print(loss.item())
num_episodes = 120
max_episode_steps=600 
tempRew = -1000
lastScore = 0
maxScore = []
reward_memory=[]
state_memory=[]
action_memory=[]
reward_all=[]
state_all=[]
for i_episode in range(num_episodes):
    # Initialize the environment and state

    print('\nStarting episode No.{}'.format(i_episode+1))
    
    state = env.reset()
    state_all.append(state)
    state = torch.from_numpy(state)
    for t in range(max_episode_steps):
        t=t + 1
        print('\nStarting time step No.{}'.format(t))
        print(tempRew)
        # Select and perform an action
        action = select_action(state)

        obs, score, done, _ = env.step(action.item())
        # record the highest score, corresponding to the highest Q factor
        if score > tempRew:
            tempRew = score
            state_best=obs
        # score = torch.tensor([score], device=device)

        # calculate the reward
        reward = score - lastScore
        print(score, reward, obs)

        reward = torch.tensor([reward], device=device)

        # Observe new state
        if not done:
            next_state = torch.from_numpy(obs)
        else:
            next_state = None
        if score >= env.spec.reward_threshold:
            print('\nSolved! Episode: {}, Steps: {}, Current_state: {}, Current_reward: {}\n'.format(
                i_episode, t, next_state, score))
            break

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        lastScore = score

        # Move to the next state
        state = next_state
        reward_memory.append(reward)
        state_memory.append(state)
        action_memory.append(action)
        # Perform one step of the optimization (on the policy network)
        # don't need to train every step
        if steps_done % UPDATE_FREQ == 0:
            optimize_model()
            writer.add_scalar('training/scores', score, steps_done)

        if done:
            break
        if t % 20==0:  
            clear_output()
    reward_all.append(tempRew)
    print('\nlargest score so far: {}'.format(tempRew))
    maxScore.append(tempRew)
    writer.add_scalar('training/max_scores', tempRew, i_episode)

    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        print('updating target network...')
        print(maxScore)
        target_net.load_state_dict(policy_net.state_dict())

    if score >= env.spec.reward_threshold:
        print('Solved! Episode: {}, Current_state: {}, Current_reward: {}\n'.format(
            i_episode, next_state, score))
        break



print('Training Complete')
writer.close()


Starting time step No.201
54.79622
tensor([[5.7373, 5.6789, 5.8354, 5.7846, 5.6440, 5.7603, 5.6378, 5.7814, 5.7882,
         5.6687, 5.7191]], device='cuda:0')
tensor([2], device='cuda:0')
Initializing eigensolver data
Computing 14 bands with 1e-07 tolerance
Working in 2 dimensions.
Grid size is 16 x 168 x 1.
Solving for 7 bands at a time.
Creating Maxwell data...
Mesh size is 3.
Lattice vectors:
     (1, 0, 0)
     (0, 10.3923, 0)
     (0, 0, 1)
Cell volume = 10.3923
Reciprocal lattice vectors (/ 2 pi):
     (1, -0, 0)
     (-0, 0.096225, -0)
     (0, -0, 1)
Geometric objects:
     cylinder, center = (-0.5,-5.19615,0)
          radius 0.322, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-0.5,-3.4641,0)
          radius 0.322, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-0.5,-1.73205,0)
          radius 0.322, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-0.5,1.73205,0)
          radius 0.322, height 1e+20, axis (0, 0, 1)
     cylinder, center = (-0.5,3.4641

tefreqs:, 1, 0.28, 0, 0, 0.28, 0.119072, 0.13493, 0.135112, 0.151753, 0.152336, 0.177354, 0.178022, 0.205322, 0.209056, 0.235849, 0.236183, 0.252516, 0.279665, 0.287127
elapsed time for k point: 0.43950724601745605
solve_kpoint (0.286286,0,0):
Solving for bands 1 to 7...
    linmin: converged after 3 iterations.
    iteration    1: trace = 0.1647384841733803 (0.0385741% change)
    linmin: converged after 3 iterations.
    iteration    2: trace = 0.1647370980240617 (0.000841428% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.1647368183749134 (0.000169755% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.1647367939792305 (1.48089e-05% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.1647367818956935 (7.33506e-06% change)
Finished solving for bands 1 to 7 after 5 iterations.
Solving for bands 8 to 14...
    linmin: converged after 3 iterations.
    iteration    1: trace = 0.4232589775060555 (0.

Finished solving for bands 1 to 7 after 5 iterations.
Solving for bands 8 to 14...
    linmin: converged after 3 iterations.
    iteration    1: trace = 0.4288808255623318 (0.027805% change)
    linmin: converged after 2 iterations.
    iteration    2: trace = 0.4288528690008611 (0.0065187% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.4288471788818233 (0.00132683% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.4288425950681188 (0.00106887% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.4288412872708843 (0.00030496% change)
    linmin: converged after 2 iterations.
    iteration    6: trace = 0.4288407779596285 (0.000118765% change)
    linmin: converged after 2 iterations.
    iteration    7: trace = 0.4288406786099438 (2.3167e-05% change)
    linmin: converged after 2 iterations.
    iteration    8: trace = 0.42884063933525 (9.15834e-06% change)
Finished solving for bands 8 to 14 after

    iteration    2: trace = 0.4326868258961324 (0.00920002% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.4326781587573463 (0.00200312% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.4326693827124994 (0.00202833% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.4326667338976312 (0.000612205% change)
    linmin: converged after 2 iterations.
    iteration    6: trace = 0.4326656226494432 (0.000256837% change)
    linmin: converged after 2 iterations.
    iteration    7: trace = 0.4326653766530461 (5.6856e-05% change)
    linmin: converged after 2 iterations.
    iteration    8: trace = 0.4326652657402468 (2.56348e-05% change)
    linmin: converged after 2 iterations.
    iteration    9: trace = 0.432665242509181 (5.36929e-06% change)
Finished solving for bands 8 to 14 after 9 iterations.
Finished k-point with 7 mean iterations/band.
tefreqs:, 10, 0.336571, 0, 0, 0.336571, 0.140058, 0.158154

    linmin: converged after 2 iterations.
    iteration    2: trace = 0.4336352092430535 (0.0106379% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.4336262292881868 (0.00207088% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.4336171040841598 (0.00210442% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.4336141196738293 (0.000688262% change)
    linmin: converged after 2 iterations.
    iteration    6: trace = 0.4336127827505171 (0.000308321% change)
    linmin: converged after 2 iterations.
    iteration    7: trace = 0.4336124424276208 (7.84855e-05% change)
    linmin: converged after 2 iterations.
    iteration    8: trace = 0.4336122770059078 (3.81497e-05% change)
    linmin: converged after 2 iterations.
    iteration    9: trace = 0.433612244400432 (7.5195e-06% change)
Finished solving for bands 8 to 14 after 9 iterations.
Finished k-point with 7.5 mean iterations/band.
tefreqs:, 14, 0

...

    linmin: converged after 2 iterations.
    iteration    4: trace = 0.4266844646809522 (0.00109572% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.4266830357292338 (0.000334897% change)
    linmin: converged after 2 iterations.
    iteration    6: trace = 0.4266824057055288 (0.000147656% change)
    linmin: converged after 2 iterations.
    iteration    7: trace = 0.4266822216700696 (4.31317e-05% change)
    linmin: converged after 2 iterations.
    switching to approximate line minimization (decrease time by 56.1111%)
    iteration    8: trace = 0.4266821067579793 (2.69315e-05% change)
    iteration    9: trace = 0.4266820813813211 (5.94744e-06% change)
Finished solving for bands 8 to 14 after 9 iterations.
Finished k-point with 9 mean iterations/band.
tefreqs:, 21, 0.405714, 0, 0, 0.405714, 0.164652, 0.184986, 0.189048, 0.196824, 0.197655, 0.21234, 0.213756, 0.22893, 0.229388, 0.235252, 0.244876, 0.249491, 0.258167, 0.2783
elapsed time for k point: 

    iteration    8: trace = 0.4208741763036781 (2.10191e-05% change)
    linmin: converged after 2 iterations.
    iteration    9: trace = 0.4208741555006398 (4.94282e-06% change)
Finished solving for bands 8 to 14 after 9 iterations.
Finished k-point with 10 mean iterations/band.
tefreqs:, 24, 0.424571, 0, 0, 0.424571, 0.171163, 0.191837, 0.196675, 0.20309, 0.204151, 0.216666, 0.218051, 0.225211, 0.230702, 0.234937, 0.242885, 0.246476, 0.257012, 0.275566
elapsed time for k point: 0.2366335391998291
solve_kpoint (0.430857,0,0):
Solving for bands 1 to 7...
    linmin: converged after 3 iterations.
    iteration    1: trace = 0.2875946426615744 (0.0435682% change)
    linmin: converged after 2 iterations.
    iteration    2: trace = 0.2875702223379881 (0.00849159% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.2875642901572906 (0.00206288% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.2875577189146748 (0.00228516% change)

    linmin: converged after 2 iterations.
    iteration   15: trace = 0.2972944394353582 (3.94005e-05% change)
    linmin: converged after 2 iterations.
    iteration   16: trace = 0.2972943715859442 (2.28223e-05% change)
    linmin: converged after 2 iterations.
    iteration   17: trace = 0.2972943378081149 (1.13617e-05% change)
    linmin: converged after 2 iterations.
    iteration   18: trace = 0.2972943135754463 (8.15107e-06% change)
Finished solving for bands 1 to 7 after 18 iterations.
Solving for bands 8 to 14...
    linmin: converged after 4 iterations.
    iteration    1: trace = 0.4143719765870146 (0.0420834% change)
    linmin: converged after 2 iterations.
    iteration    2: trace = 0.4143256474010141 (0.0111812% change)
    linmin: converged after 2 iterations.
    iteration    3: trace = 0.4143194031695162 (0.00150709% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.4143153003318369 (0.000990265% change)
    linmin: converged after 2 ite

    linmin: converged after 2 iterations.
    iteration    3: trace = 0.3086299521711602 (0.00332809% change)
    linmin: converged after 2 iterations.
    iteration    4: trace = 0.3086184949500745 (0.00371235% change)
    linmin: converged after 2 iterations.
    iteration    5: trace = 0.3086126440790685 (0.00189584% change)
    linmin: converged after 2 iterations.
    iteration    6: trace = 0.3086066367663739 (0.00194657% change)
    linmin: converged after 2 iterations.
    iteration    7: trace = 0.308604176650117 (0.000797172% change)
    linmin: converged after 2 iterations.
    switching to approximate line minimization (decrease time by 52.2757%)
    iteration    8: trace = 0.3086028519796769 (0.000429247% change)
    switching back to exact line minimization
    iteration    9: trace = 0.3086024122546509 (0.000142489% change)
    linmin: converged after 2 iterations.
    iteration   10: trace = 0.3086020465249463 (0.000118512% change)
    linmin: converged after 2 iteratio

SystemError: <built-in function mode_solver_solve_kpoint> returned a result with an error set

In [14]:
print(state_best)

[ 0.041  0.143  0.095 -0.222  0.001]


In [15]:
tempRew

54.79622

In [16]:
state_memory

[tensor([ 0.0210,  0.1510,  0.0480, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1500,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1500,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1500,  0.0470, -0.1300, -0.0090]),
 tensor([ 0.0210,  0.1500,  0.0470, -0.1300, -0.0090]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1300, -0.0090]),
 tensor([ 0.0210,  0.1520,  0.0470, -0.1300, -0.0090]),
 tensor([ 0.0210,  0.1520,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0090]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0080]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0070]),
 tensor([ 0.0210,  0.1510,  0.0470, -0.1290, -0.0080]),
 tensor([ 0.0210,  0.1520,  0.0470, -0.1290, -0.0080]),
 tensor([ 0.0200,  0.1520,  0.0470, -0.1290, -0.0080]),
 tensor([ 0.0200,  0.1510,  0.0470, -0.1290, -0.0080]),
 tensor([ 0.0200,  0.1500,  0.0470, -0.1290, -0.

In [17]:
state_all

[array([ 0.021,  0.151,  0.048, -0.129, -0.009], dtype=float32),
 array([ 0.041,  0.142,  0.095, -0.212,  0.001], dtype=float32),
 array([ 0.027,  0.151,  0.055, -0.154, -0.009], dtype=float32),
 array([ 0.15 ,  0.147,  0.053, -0.226,  0.012], dtype=float32),
 array([ 0.002,  0.159,  0.006, -0.222, -0.001], dtype=float32),
 array([ 0.106,  0.178,  0.098, -0.083, -0.007], dtype=float32),
 array([ 0.063,  0.167,  0.078, -0.128,  0.001], dtype=float32),
 array([ 0.081,  0.092,  0.097, -0.14 , -0.008], dtype=float32),
 array([ 0.027,  0.096,  0.04 , -0.219,  0.004], dtype=float32),
 array([ 0.054,  0.175,  0.002, -0.169, -0.001], dtype=float32),
 array([ 0.105,  0.164,  0.084, -0.113,  0.004], dtype=float32),
 array([ 0.015,  0.173,  0.032, -0.173, -0.009], dtype=float32),
 array([ 0.059,  0.09 ,  0.067, -0.222, -0.005], dtype=float32),
 array([ 0.042,  0.112,  0.04 , -0.19 , -0.014], dtype=float32),
 array([ 0.068,  0.17 ,  0.026, -0.149,  0.007], dtype=float32),
 array([ 0.047,  0.118,  

In [18]:
tempRew

54.79622

In [10]:
f = open("tm2.csv",'a+',encoding='utf-8')   
csv_writer = csv.writer(f)
csv_writer.writerow(tm_freqs[:,5])
f.close()

NameError: name 'tm_freqs' is not defined

In [11]:
import pandas as pd
pd.options.display.max_rows = 2000
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [19]:
reward_all

[5.0,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.79622,
 54.

In [ ]:
memory=[20,20,58.147,58.147,58.147]

In [ ]:
#可以改一下优化器，改为RMSprop.。给memory.push加一些条件 增加经验池数量。考虑加工，对结构大小加以限制